# DATA CLEANING AND PREPERATION

In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

data = pd.read_csv('japandata.csv')

In [56]:
# check for any duplicates + removing
duplicate_rows = data.duplicated().sum()
print("Number of duplicate rows: ", duplicate_rows)

if duplicate_rows > 0:
    data = data.drop_duplicates()

Number of duplicate rows:  0


# Feature editing 

In [57]:
# converting 'time' from string to datetime
data['time'] = pd.to_datetime(data['time'])

# Feature Engineering (year used during one of the cleaning steps so we did this early)
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month
data['day'] = data['time'].dt.day
data['mag_category'] = pd.cut(data['mag'], bins=[0, 4.9, 5.9, 6.9, 10], labels=[1, 2, 3, 4])

scaler = MinMaxScaler()
data['normalized_magnitude'] = scaler.fit_transform(data[['mag']])

# season function to add as feature in new dataset
def get_season(month):
    if 3 <= month <= 5:
        return 1
    elif 6 <= month <= 8:
        return 2
    elif 9 <= month <= 11:
        return 3
    else:  # December to February
        return 4

data['season'] = data['month'].apply(get_season)


# Dropping features we dont need
data = data.drop(['magType', 'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type', 'status', 'locationSource', 'magSource'], axis=1)

# Data Cleaning

In [58]:
# dropping rows without mag (its an important feature + only hanful missing)
data = data.dropna(subset=['mag'])

# outlier detection for 'depth, magError, depthError, horizontalError'
depth_q1 = data['depth'].quantile(0.25)
depth_q3 = data['depth'].quantile(0.75)
depth_iqr = depth_q3 - depth_q1
depth_lower_bound = depth_q1 - 1.5 * depth_iqr
depth_upper_bound = depth_q3 + 1.5 * depth_iqr
data = data[
    (data['depth'].isna()) | 
    ((data['depth'] >= depth_lower_bound) & (data['depth'] <= depth_upper_bound))
]

magError_q1 = data['magError'].quantile(0.25)
magError_q3 = data['magError'].quantile(0.75)
magError_iqr = magError_q3 - magError_q1
magError_lower_bound = magError_q1 - 1.5 * magError_iqr
magError_upper_bound = magError_q3 + 1.5 * magError_iqr
data = data[
    (data['magError'].isna()) | 
    ((data['magError'] >= magError_lower_bound) & (data['magError'] <= magError_upper_bound))
]

depthError_q1 = data['depthError'].quantile(0.25)
depthError_q3 = data['depthError'].quantile(0.75)
depthError_iqr = depthError_q3 - depthError_q1
depthError_lower_bound = depthError_q1 - 1.5 * depthError_iqr
depthError_upper_bound = depthError_q3 + 1.5 * depthError_iqr
data = data[
    (data['depthError'].isna()) | 
    ((data['depthError'] >= depthError_lower_bound) & (data['depthError'] <= depthError_upper_bound))
]

horizontalError_q1 = data['horizontalError'].quantile(0.25)
horizontalError_q3 = data['horizontalError'].quantile(0.75)
horizontalError_iqr = horizontalError_q3 - horizontalError_q1
horizontalError_lower_bound = horizontalError_q1 - 1.5 * horizontalError_iqr
horizontalError_upper_bound = horizontalError_q3 + 1.5 * horizontalError_iqr
data = data[
    (data['horizontalError'].isna()) | 
    ((data['horizontalError'] >= horizontalError_lower_bound) & (data['horizontalError'] <= horizontalError_upper_bound))
]



# handling missing values
# depth
data['depth'] = data['depth'].fillna(data['depth'].median())


# magNst
data['time_group'] = data['year'].apply(lambda x: 'before_2011' if x <= 2010 else 'after_2011')
medians = data.groupby('time_group')['magNst'].transform('median')
data = data.drop('time_group', axis=1)
data['magNst'] = data['magNst'].fillna(medians)

# nst
median_nst = data.loc[data['year'] <= 2013, 'nst'].median()
data['nst'] = data['nst'].fillna(median_nst)

# the Errors
columns_to_fill = ['horizontalError', 'depthError', 'magError']
data[columns_to_fill] = data[columns_to_fill].fillna(value=pd.NA)



# final check for missing values
print(data.isnull().sum())

time                       0
latitude                   0
longitude                  0
depth                      0
mag                        0
nst                        0
horizontalError         6743
depthError              2931
magError                6460
magNst                     0
year                       0
month                      0
day                        0
mag_category               0
normalized_magnitude       0
season                     0
dtype: int64


## For error-related features, we're leaving them as NAN to indicate uncertainty. We also didnt want to fill it in as they were a big chunk of data

# Data integrity check

In [59]:
assert (data['depth'] >= 0).all(), "Depth contains negative values"
assert (data['mag'] >= 0).all(), "Magnitude contains negative values"
assert (data['latitude'].between(-90, 90)).all(), "Latitude out of bounds"
assert (data['longitude'].between(-180, 180)).all(), "Longitude out of bounds"

# Saving new data to different file

In [60]:
data.to_csv('cleaned_japandata.csv', index=False)

# Checking final number of rows and colums to physically compare with the new cleaned data

In [61]:
num_rows = data.shape[0]
print("Number of rows:", num_rows)

num_cols = data.shape[1]
print("Number of cols:", num_cols)

Number of rows: 9607
Number of cols: 16
